## 순환 신경망 (Recurrent Neural Network, RNN)

순환 신경망(RNN)은 시퀀스 데이터 처리에 주로 사용되는 신경망 구조입니다. RNN은 이전 시점의 출력을 현재 시점의 입력으로 사용하며, 시퀀스 데이터에서 순서와 의존성을 고려하여 정보를 처리합니다. RNN은 순차적인 특징을 가진 데이터에 적합하며, 자연어 처리, 음성 인식, 시계열 데이터 등 다양한 작업에 활용됩니다.

---

이번 실습에서는 RNN을 사용하여 텍스트 데이터를 학습하고, 텍스트 생성 작업을 수행합니다. RNN 모델을 구축하고 훈련하는 과정을 진행한 후, 모델을 사용하여 새로운 텍스트를 생성합니다.

실습에서는 데이터 전처리, 모델 구축, 모델 컴파일, 훈련, 텍스트 생성 등의 과정을 진행합니다. RNN을 사용하여 순차적인 데이터를 처리하고 텍스트를 생성하는 과정을 경험해보세요.

---


### 1. 필요한 라이브러리 임포트 하기

__사내 PC 실습 안내__ 
- 본 실습자료는 외부 인터넷 환경에서 자료를 다운로드하는 과정이 담겨 있습니다.
- 사내 PC에서 실습하실 경우 SSL 인증 오류가 발생할 수 있습니다.
- SSL 인증을 생략하여 오류를 방지하는 아래 코드를 실행한 후, 실습을 진행 해 주시면 됩니다.
- 따라서, 내용은 영상 속 강의자료와 다를 수 있습니다.

```python
import ssl
# https 인증 시, 기본 인증 컨텍스트를, 인증 생략 컨텍스트로 변경
ssl._create_default_https_context = ssl._create_unverified_context
```


In [ ]:
import ssl
# https 인증 시, 기본 인증 컨텍스트를, 인증 생략 컨텍스트로 변경
ssl._create_default_https_context = ssl._create_unverified_context

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 

- `import numpy as np`: 수치 계산을 위한 라이브러리를 가져오는 구문입니다. `numpy`는 다차원 배열 처리와 수학 함수를 제공하는 라이브러리입니다.

- `import tensorflow as tf`: TensorFlow 라이브러리를 가져오는 구문입니다. TensorFlow는 기계 학습과 딥러닝을 위한 오픈 소스 프레임워크로, 신경망 모델을 구축하고 학습시키는 데 사용됩니다.

- `from tensorflow.keras.datasets import imdb`: TensorFlow의 `keras.datasets` 모듈에서 `imdb` 데이터셋을 가져오는 구문입니다. `imdb` 데이터셋은 영화 리뷰에 대한 텍스트 데이터와 이에 대한 긍정 또는 부정의 레이블로 구성된 데이터셋입니다.

- `from tensorflow.keras.models import Sequential`: TensorFlow의 `keras.models` 모듈에서 `Sequential` 클래스를 가져오는 구문입니다. `Sequential` 클래스는 순차적인 신경망 모델을 생성하기 위한 클래스로, 여러 개의 레이어를 선형으로 쌓아 구성할 수 있습니다.

- `from tensorflow.keras.layers import Embedding, LSTM, Dense`: TensorFlow의 `keras.layers` 모듈에서 `Embedding`, `LSTM`, `Dense` 등의 레이어 클래스를 가져오는 구문입니다. 
    - `Embedding`은 단어 임베딩을 위한 레이어로, 단어를 밀집 벡터로 표현하는 데 사용됩니다.
    - `LSTM`은 장기 의존성을 학습하기 위한 순환 신경망 레이어로, 순차 데이터를 처리하는 데 사용됩니다.
    - `Dense`는 완전 연결 레이어로, 입력과 출력을 모두 연결하여 가중치를 학습하는 데 사용됩니다.

- `from sklearn.preprocessing import MinMaxScaler`: scikit-learn의 `preprocessing` 모듈에서 `MinMaxScaler` 클래스를 가져오는 구문입니다. `MinMaxScaler`는 데이터를 정규화하는 데 사용되며, 입력 데이터를 특정 범위로 스케일링하여 모델의 학습을 돕는 데 사용됩니다.

- `from sklearn.model_selection import train_test_split`: scikit-learn의 `model_selection` 모듈에서 `train_test_split` 함수를 가져오는 구문입니다. `train_test_split` 함수는 데이터셋을 훈련 세트와 테스트 세트로 나누는 데 사용되며, 모델의 훈련과 일반화 성능 평가에 활용됩니다.


### 2. 데이터셋 생성
##### IMDB 영화 리뷰 데이터셋 로드

In [ ]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

### 3. 데이터 전처리

- `tf.keras.preprocessing.sequence.pad_sequences`: 시퀀스 데이터를 패딩하는 함수입니다. 시퀀스 데이터는 각각 길이가 다를 수 있으며, 신경망 모델에 입력으로 전달하기 위해서는 고정된 길이로 맞춰주는 것이 필요합니다. `pad_sequences` 함수는 이를 위해 시퀀스 데이터에 패딩을 추가하여 모든 시퀀스의 길이를 동일하게 만들어 줍니다.

  - `sequences`: 패딩할 시퀀스 데이터를 입력으로 받습니다. 보통 2차원 배열의 형태로 주어집니다.
  
  - `maxlen`: 시퀀스의 최대 길이를 지정합니다. 이 값으로 모든 시퀀스를 패딩하여 동일한 길이로 만듭니다. 길이가 maxlen보다 작은 시퀀스는 0으로 채워진 패딩이 추가됩니다.
  
  - `dtype`: 생성된 패딩 배열의 데이터 타입을 지정합니다.
  
  - `padding`: 'pre' 또는 'post' 값을 사용하여 패딩을 추가하는 위치를 지정합니다. 'pre'는 시퀀스 앞에 패딩을 추가하고, 'post'는 시퀀스 뒤에 패딩을 추가합니다.
  
  - `truncating`: 'pre' 또는 'post' 값을 사용하여 시퀀스가 maxlen보다 긴 경우 어느 부분을 잘라낼지 지정합니다. 'pre'는 시퀀스 앞부분을 잘라내고, 'post'는 시퀀스 뒷부분을 잘라냅니다.


In [ ]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=200)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=200)

### 4. 모델(RNN) 구축
##### 전체 데이터셋을 80%의 훈련 데이터와 20%의 테스트 데이터로 나누기

In [ ]:
model = Sequential()
model.add(Embedding(10000, 128))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

- `model = Sequential()`: `Sequential` 모델을 생성합니다. `Sequential` 모델은 레이어를 선형으로 쌓아 구성할 수 있는 모델입니다.

- `model.add(Embedding(10000, 128))`: `Embedding` 레이어를 모델에 추가합니다. `Embedding` 레이어는 텍스트나 시퀀스 데이터를 밀집된 벡터 형태로 임베딩하는 역할을 합니다. 이 때, 첫 번째 인자는 입력 벡터의 크기를 나타내며, 두 번째 인자는 임베딩 벡터의 차원을 나타냅니다.

- `model.add(LSTM(128))`: `LSTM` 레이어를 모델에 추가합니다. `LSTM` 레이어는 장기 의존성(Long Short-Term Memory)을 학습할 수 있는 순환 신경망 레이어입니다. 이 레이어는 시퀀스 데이터를 입력으로 받아 시퀀스 데이터의 특징을 추출하고, 은닉 상태를 유지하며 출력을 생성합니다. 여기서 128은 LSTM 레이어의 출력 차원을 나타냅니다.

- `model.add(Dense(1, activation='sigmoid'))`: `Dense` 레이어를 모델에 추가합니다. `Dense` 레이어는 완전 연결(fully connected)된 신경망 레이어입니다. 이 레이어는 입력과 출력을 모두 연결하고, 가중치를 학습합니다. 여기서 1은 출력 뉴런의 개수를 나타내며, `sigmoid` 활성화 함수를 사용하여 이진 분류를 수행합니다.


### 5. 모델 컴파일

- `optimizer='adam'`:
  - *목적*: 모델의 최적화 알고리즘을 설정합니다.
  - *설명*: Adam(Optimization Algorithm with Adaptive Moment Estimation)은 경사 하강법의 한 종류로, 학습률을 자동으로 조정하면서 모델 파라미터를 업데이트합니다.

- `loss='binary_crossentropy'`:
  - *목적*: 손실 함수(loss function)를 설정합니다.
  - *설명*: binary_crossentropy는 이진 분류(binary classification) 모델에서 주로 사용되는 손실 함수입니다. 이 함수는 실제 클래스와 예측 클래스 간의 크로스 엔트로피 손실을 계산하여 모델이 예측을 개선하도록 돕니다.


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### 6. 모델 훈련

- `X_train`: 모델의 훈련에 사용할 입력 데이터입니다.
- `y_train`: 모델의 훈련에 사용할 타깃(레이블) 데이터입니다.
- `epochs`: 모델의 훈련 에포크(Epoch) 수를 나타냅니다. 에포크는 전체 데이터셋을 한 번 훈련하는 단위를 말하며, 에포크 수가 증가할수록 모델이 더 많이 훈련됩니다.
- `batch_size`: 모델의 훈련에 사용할 배치 크기를 나타냅니다. 배치 크기는 한 번의 반복에서 사용되는 샘플의 개수를 말하며, 모델의 가중치 업데이트는 각 배치마다 수행됩니다.
- `validation_data`: 검증 데이터셋을 지정합니다. 검증 데이터셋은 훈련 과정에서 모델의 성능을 평가하는 데 사용됩니다. 검증 데이터셋을 지정하면 각 에포크의 훈련 손실과 검증 손실, 훈련 정확도와 검증 정확도가 함께 기록됩니다.

In [ ]:
model.fit(X_train, y_train, epochs=3, batch_size=64, validation_data=(X_test, y_test))

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=8)

### 7. 모델 평가
##### 테스트 데이터(X_test, y_test)를 사용하여 모델의 손실과 정확도를 평가합니다.

- `model.evaluate(X_test, y_test)`:
  - **목적**: 모델의 성능을 평가하기 위한 메서드입니다.
  - **설명**: `evaluate` 메서드는 모델이 주어진 테스트 데이터셋을 기반으로 예측 결과와 실제 값 사이의 오차를 계산하여 모델의 성능을 평가합니다. 이 메서드는 테스트 데이터셋에 대한 예측 정확도(accuracy)나 손실(loss) 값을 반환합니다. 평가 결과를 통해 모델이 얼마나 잘 일반화되었는지 확인할 수 있습니다. 
  - **매개변수**:
    - `X_test`: 평가에 사용될 테스트 데이터의 입력값을 나타냅니다.
    - `y_test`: 평가에 사용될 테스트 데이터의 실제 출력값을 나타냅니다.

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

### 8. 예측 수행
##### imdb 데이터를 사용하여 예측 수행

In [ ]:
# 새로운 리뷰 예측
new_review = "This movie was fantastic! Highly recommended!"
word_to_index = imdb.get_word_index()

# 텍스트 전처리 및 시퀀스 패딩
new_review = new_review.lower().split()
new_review = [word_to_index[word] + 3 if word in word_to_index and word_to_index[word] + 3 < 10000 else 2 for word in new_review]
new_review = tf.keras.preprocessing.sequence.pad_sequences([new_review], maxlen=200)

# 예측 수행
prediction = model.predict(new_review)[0][0]

# 감성 예측 결과 출력
print("New Review Prediction:")
if prediction >= 0.5:
    print("Positive")
else:
    print("Negative")


## 실습 완료: RNN(Recurrent Neural Network, RNN) Summary

1. RNN 모델을 사용하여 텍스트 데이터를 학습하고 텍스트 생성 작업을 완료했습니다. RNN은 순차적인 특징을 가진 데이터에 효과적으로 사용되는 신경망 구조입니다.

2. 실습을 통해 RNN 모델의 구성과 파라미터 조정, 텍스트 데이터의 전처리 과정 등을 경험하였습니다. 학습된 모델을 사용하여 새로운 텍스트를 생성할 수 있었습니다.

3. RNN을 이용한 텍스트 생성은 다양한 자연어 처리 문제에 활용할 수 있으며, 더 많은 실습과 응용을 통해 RNN 모델을 발전시키고 다양한 텍스트 데이터에 적용해 보시기 바랍니다.